# Docker-compose oraz dev-owy projekt w Django

Marek Piechula

kato.py, 2016-03-22

# Plan

* Po co dev-owi wirtualki
* Moja historia
* Docker compose
* Przykład
* Pytania

# Po co dev-owi wirtualki

* Bo nie potrafi uruchomić aplikacji u siebie
* … albo jest za dużo do ustawiania
* Chce by jego (i nie tylko) było środowisko zbliżone do produkcji
* Nie chce zaśmiecać swojego systemu przeróżnym softem

# Virtualbox
![image](virtualbox.png)

# Vagrant

![image](vagrant.png)

![image](vagrant-fail.png)

# Docker

In [9]:
%%sh
docker run -d -p 8081:80 coreos/apache /usr/sbin/apache2ctl -v ~/hs/:/www/ -D FOREGROUND

60181320888b6f111a08047c8c132f3a5ea059419b8d8d50c919edd316d0265c


Unable to find image 'coreos/apache:latest' locally
latest: Pulling from coreos/apache
d634beec75db: Pulling fs layer
8b877d3adb63: Pulling fs layer
95dde7ac21e8: Pulling fs layer
7aaa39173c60: Pulling fs layer
d634beec75db: Verifying Checksum
d634beec75db: Download complete
8b877d3adb63: Verifying Checksum
8b877d3adb63: Download complete
d634beec75db: Pull complete
8b877d3adb63: Pull complete
7aaa39173c60: Verifying Checksum
7aaa39173c60: Download complete
95dde7ac21e8: Verifying Checksum
95dde7ac21e8: Download complete
95dde7ac21e8: Pull complete
7aaa39173c60: Pull complete
Digest: sha256:9af520cee7bedcda564970ff790cdf2e72b6daccce8539f6b3c880ed7fc21091
Status: Downloaded newer image for coreos/apache:latest


In [13]:
%%sh
curl localhost:8081

<html><body><h1>It works!</h1>
<p>This is the default web page for this server.</p>
<p>The web server software is running but no content has been added, yet.</p>
</body></html>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   177  100   177    0     0  35371      0 --:--:-- --:--:-- --:--:-- 44250


In [14]:
%%sh
docker ps

CONTAINER ID        IMAGE               COMMAND                  CREATED              STATUS              PORTS                  NAMES
06bdc0a504a3        coreos/apache       "/usr/sbin/apache2ctl"   About a minute ago   Up About a minute   0.0.0.0:8081->80/tcp   focused_bohr


# Docker Compose

### docker-compose.yml

In [ ]:
db:
  image: postgres
  environment:
    POSTGRES_USER: socek
    POSTGRES_PASSWORD: kocham-marka-123

web:
  build: .
  command: python manage.py runserver 0.0.0.0:8000
  volumes:
    - ./webapp/:/code/
    - ./webapp/static/:/static/
    - ./webapp/media/:/media/
  ports:
    - "8000:8000"
  links:
    - db

### Dockerfile

In [ ]:
FROM python:3.4
ENV PYTHONUNBUFFERED 1
ENV DJANGO_CONFIGURATION Docker
ENV DJANGO_SETTINGS_MODULE mayzla.settings
RUN mkdir /code
ADD webapp/requirements.txt /code/
RUN pip install -r /code/requirements.txt --upgrade
ADD webapp /code/
WORKDIR /code

In [3]:
%%sh
cd /home/firemark/git/calendar-oswiecim/
docker-compose build web

Step 1 : FROM python:3.4
 ---> f800fade39e3
Step 2 : ENV PYTHONUNBUFFERED 1
 ---> Using cache
 ---> efa7ffbf5430
Step 3 : ENV DJANGO_CONFIGURATION Docker
 ---> Using cache
 ---> 0b4167dee740
Step 4 : ENV DJANGO_SETTINGS_MODULE mayzla.settings
 ---> Using cache
 ---> cb505245302f
Step 5 : RUN mkdir /code
 ---> Using cache
 ---> cc7905a0ac04
Step 6 : ADD webapp/requirements.txt /code/
 ---> Using cache
 ---> ab6cf56a9844
Step 7 : RUN pip install -r /code/requirements.txt --upgrade
 ---> Using cache
 ---> d57f1a84c998
Step 8 : ADD webapp /code/
 ---> 2efdcc7fce47
Removing intermediate container a1d7d992e531
Step 9 : WORKDIR /code
 ---> Running in 0c8c912fc5fc
 ---> e947f6a077b0
Removing intermediate container 0c8c912fc5fc
Successfully built e947f6a077b0


Building web


In [ ]:
docker-compose up web

In [ ]:
Creating calendaroswiecim_db_1
Creating calendaroswiecim_web_1
Attaching to calendaroswiecim_web_1
web_1 | Performing system checks...
web_1 | 
web_1 | System check identified no issues (0 silenced).
web_1 | 
web_1 | You have unapplied migrations; your app may not work properly until they are applied.
web_1 | Run 'python manage.py migrate' to apply them.
web_1 | March 21, 2016 - 00:48:29
web_1 | Django version 1.8.8, using settings 'mayzla.settings'
web_1 | Starting development server at http://0.0.0.0:8000/
web_1 | Quit the server with CONTROL-C.

### settings.py

In [ ]:
from configurations import Configuration

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))

class Base(Configuration):
    SECRET_KEY = 'lol :)'
    ALLOWED_HOSTS = []
    INSTALLED_APPS = (
    # … … …

class Dev(Base):
    STATIC_ROOT = os.path.join(BASE_DIR, 'static')
    MEDIA_ROOT = os.path.join(BASE_DIR, 'media')
    DBBACKUP_STORAGE_OPTIONS = {'location': '/tmp/'}
    DATABASES = {'default': dj_database_url.config(default='sqlite:///db.sqlite3')}
    DEBUG = True

class Docker(Base):
    STATIC_ROOT = '/static/'
    MEDIA_ROOT = '/media/'
    DBBACKUP_STORAGE_OPTIONS = {'location': '/tmp/'}
    DATABASES = {'default': dj_database_url.config(
        default='postgres://socek:kocham-marka-123@db/socek'
    )}
    DEBUG = True

In [ ]:
docker-compose run web ./manage.py migrate       

In [ ]:
Operations to perform:
  Synchronize unmigrated apps: staticfiles, widget_tweaks, thumbnail, messages, ckeditor, bootstrap3, dbbackup, osm_field, easy_select2, datetimewidget
  Apply all migrations: auth, contenttypes, admin, calendars, sessions
Synchronizing apps without migrations:
  Creating tables...
    Running deferred SQL...
  Installing custom SQL...
Running migrations:
  Rendering model states... DONE
  Applying calendars.0024_auto_20160220_1621... OK
  Applying calendars.0025_auto_20160222_1936... OK
  Applying calendars.0026_auto_20160222_1953... OK
  Applying calendars.0027_organizer_url... OK
  Applying calendars.0028_auto_20160225_2336... OK

#Demo

#Q & A

* https://github.com/firemark/notebooks/tree/master/docker-compose
* https://github.com/firemark/calendar-oswiecim/tree/docker